In [7]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from srcnn_model import build_srcnn
from tensorflow.keras.models import load_model

HR_DIR = "HR1"
LR_DIR = "LR2"
RESULTS_DIR = "results"
SRCNN_WEIGHTS = "models/srcnn_trained.weights_new.h5"
TOP_N = 20
MAX_IMAGES = 100

os.makedirs(f"{RESULTS_DIR}/plots", exist_ok=True)

def load_images():
    hr_list = sorted(os.listdir(HR_DIR))[:MAX_IMAGES]
    lr_list = sorted(os.listdir(LR_DIR))[:MAX_IMAGES]
    return hr_list, lr_list

def upscale_bicubic(img, shape):
    return cv2.resize(img, (shape[1], shape[0]), interpolation=cv2.INTER_CUBIC)

def evaluate(img1, img2):
    return psnr(img1, img2, data_range=1.0), ssim(img1, img2, channel_axis=2, data_range=1.0)

def plot_and_save(lr, bicubic, srcnn, hr, name, scores, save_dir):
    methods = [("Bicubic", bicubic), ("SRCNN", srcnn), ("Original", hr)]
    plt.figure(figsize=(18, 6))
    for i, (title, img) in enumerate(methods):
        plt.subplot(1, 3, i + 1)
        plt.imshow(np.clip(img, 0, 1))
        if title != "Original":
            psnr_val, ssim_val = scores[title]
            plt.title(f"{title}\nPSNR: {psnr_val:.2f}, SSIM: {ssim_val:.4f}")
        else:
            plt.title("Original")
        plt.axis('off')
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir, f"{name}.png"))
    plt.close()

def run_benchmark():
    hr_list, lr_list = load_images()
    model = build_srcnn()
    model.load_weights(SRCNN_WEIGHTS)

    metrics = []

    for i, (hr_file, lr_file) in enumerate(tqdm(zip(hr_list, lr_list), total=len(hr_list))):
        hr = cv2.imread(os.path.join(HR_DIR, hr_file))
        lr = cv2.imread(os.path.join(LR_DIR, lr_file))

        hr = cv2.cvtColor(hr, cv2.COLOR_BGR2RGB) / 255.0
        lr = cv2.cvtColor(lr, cv2.COLOR_BGR2RGB) / 255.0

        lr_up = upscale_bicubic(lr, hr.shape)
        lr_input = np.expand_dims(lr_up, axis=0)
        sr = model.predict(lr_input)[0]
        sr = np.clip(sr, 0, 1)

        psnr_bicubic, ssim_bicubic = evaluate(hr, lr_up)
        psnr_srcnn, ssim_srcnn = evaluate(hr, sr)

        plot_and_save(lr_up, lr_up, sr, hr, f"comparison_{i}", {
            "Bicubic": (psnr_bicubic, ssim_bicubic),
            "SRCNN": (psnr_srcnn, ssim_srcnn)
        }, f"{RESULTS_DIR}/plots")

        metrics.append({
            "image": hr_file,
            "PSNR_Bicubic": psnr_bicubic,
            "SSIM_Bicubic": ssim_bicubic,
            "PSNR_SRCNN": psnr_srcnn,
            "SSIM_SRCNN": ssim_srcnn
        })

    df = pd.DataFrame(metrics)
    df.to_csv(os.path.join(RESULTS_DIR, "metrics.csv"), index=False)
    print("\nSaved metrics to results/metrics.csv")

    # Save Top-N by PSNR_SRCNN
    top_df = df.sort_values(by="PSNR_SRCNN", ascending=False).head(TOP_N)
    top_df.to_csv(os.path.join(RESULTS_DIR, f"top_{TOP_N}_srcnn.csv"), index=False)
    print(f"Top {TOP_N} results saved.")

if __name__ == "__main__":
    run_benchmark()


ValueError: Cannot assign value to variable ' conv2d_9/kernel:0': Shape mismatch.The variable shape (9, 9, 3, 128), and the assigned value shape (128, 1, 9, 9) are incompatible.